In [ ]:
!pip install langchain langchain_community langgraph langchain_groq

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached langgraph-0.6.2-py3-none-any.whl.metadata (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 971.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 18.1 MB/s eta 0:00:00


In [ ]:
from typing import List, Dict
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from google.colab import userdata

groq_api_key = userdata.get("GROQ_API_KEY")

In [ ]:
#Step1 : Define State
class State(Dict):
  messages: List[Dict[str, str]]

#Step2 : Initialize StateGraph
graph_builder = StateGraph(State)

#Initalize the LLM
llm = ChatGroq(model= "gemma2-9b-it", temperature= 0, api_key= groq_api_key)

#Define chatbot function
def chatbot(state:State):
  response= llm.invoke(state["messages"])
  state["messages"].append({"role": "assitant", "content": response})
  return {"messages": state["messages"]}


#Add Nodes and Edges
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


#Compile the graph
graph= graph_builder.compile()

#Stream updates
def stream_graph_updates(user_input: str):
  state = {"messages": [{"role": "user", "content": user_input}]}
  for event in graph.stream(state):
    for value in event.values():
      print("Assistant:", value["messages"][-1]["content"])


#Run the chatbot in a loop
if __name__=="__main__":
  while True:
    try:
      user_input = input("User: ")
      if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodabye!!")
        break

      stream_graph_updates(user_input)
    except Exception as e:
      print(f"An error ocurred: {e}")
      break



User: Hiii
Assistant: content='Hello! 👋  How can I help you today? 😊\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 11, 'total_tokens': 26, 'completion_time': 0.027272727, 'prompt_time': 0.001177509, 'queue_time': 0.251676391, 'total_time': 0.028450236}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--544c459d-bad9-497d-9745-30cdef3731d1-0' usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26}
User: Tell me about Agentic AI
Assistant: content="Agentic AI refers to a type of artificial intelligence that exhibits **autonomous decision-making and goal-oriented behavior**. \n\nHere's a breakdown of key characteristics:\n\n**1. Autonomy:** Agentic AI systems can operate independently, making decisions and taking actions without constant human intervention. They have the ability to perceive their environment, interp